In [4]:
from docling.document_converter import DocumentConverter
from dotenv import dotenv_values
import os
from pathlib import Path



In [5]:
# set the environment variables
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
ENV_VARIABLES = {
    **dotenv_values(BASE_DIR / ".env"),
    **os.environ
}

In [8]:
source = DATA_DIR / "INFORME SPLIT NVIDIA.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())  # output: "## Docling Technical Report[...]"

Ba

<!-- image -->

xtb

## SPLIT NVIDIA XTB LATAM

## NVIDIA:

Cuando  analizamos  las  acciones  de  mejor  rendimiento  en  lo  que  va  del  año, observamos  que NVIDIA  destaca  con  un  aumento  del  138% , lo  que  contrasta significativamente con el rendimiento del índice de referencia S&amp;P 500, que ha tenido un incremento de +11.40%.

NVIDIA, fundada en 1993, revolucionó el mercado tecnológico con la invención de la GPU en 1999, lo que impulsó el crecimiento del mercado de videojuegos para PC y redefinió los gráficos por computadora. En 2024, NVIDIA se ha consolidado como líder  en  el  sector  de  la  inteligencia  artificial  (IA) y  la  computación  acelerada.  La compañía ha superado a gigantes tecnológicos, llegando a ser la cuarta empresa más valiosa de EE.UU . con una capitalización de mercado de $1.82 billones.

La importancia de NVIDIA en el sector tecnológico radica en su dominio del mercado de  chips  de  IA ,  con  una  participación  del  83%  en  este  segment

In [10]:
markdown_variable_docling = result.document.export_to_markdown()
print(markdown_variable_docling)

#save the string to a markdown file
with open(DATA_DIR / "INFORME SPLIT NVIDIA_docling.md", "w", encoding="utf-8") as f:
    f.write(markdown_variable_docling)

Ba

<!-- image -->

xtb

## SPLIT NVIDIA XTB LATAM

## NVIDIA:

Cuando  analizamos  las  acciones  de  mejor  rendimiento  en  lo  que  va  del  año, observamos  que NVIDIA  destaca  con  un  aumento  del  138% , lo  que  contrasta significativamente con el rendimiento del índice de referencia S&amp;P 500, que ha tenido un incremento de +11.40%.

NVIDIA, fundada en 1993, revolucionó el mercado tecnológico con la invención de la GPU en 1999, lo que impulsó el crecimiento del mercado de videojuegos para PC y redefinió los gráficos por computadora. En 2024, NVIDIA se ha consolidado como líder  en  el  sector  de  la  inteligencia  artificial  (IA) y  la  computación  acelerada.  La compañía ha superado a gigantes tecnológicos, llegando a ser la cuarta empresa más valiosa de EE.UU . con una capitalización de mercado de $1.82 billones.

La importancia de NVIDIA en el sector tecnológico radica en su dominio del mercado de  chips  de  IA ,  con  una  participación  del  83%  en  este  segment

### Figure export

In [6]:
import logging
import time
from pathlib import Path
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [10]:
_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 2.0


def document_to_markdown_images():
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path(DATA_DIR / "auge-de-los-llm.pdf")
    output_dir = Path(DATA_DIR)

    # Important: For operating with page images, we must keep them, otherwise the DocumentConverter
    # will destroy them for cleaning up memory.
    # This is done by setting PdfPipelineOptions.images_scale, which also defines the scale of images.
    # scale=1 correspond of a standard 72 DPI image
    # The PdfPipelineOptions.generate_* are the selectors for the document elements which will be enriched
    # with the image field
    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    # Save page images
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")

    # Save images of figures and tables
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

    # Save markdown with embedded pictures
    # md_filename = output_dir / f"{doc_filename}-with-images.md"
    # conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)


    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")


In [11]:
document_to_markdown_images()

INFO:docling.document_converter:Going to convert document batch...
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.pipeline.base_pipeline:Processing document auge-de-los-llm.pdf
INFO:docling.document_converter:Finished converting document auge-de-los-llm.pdf in 136.03 sec.
INFO:__main__:Document converted and figures exported in 153.11 seconds.
